## <font color='green'>Rerun the cell below</font>

In [1]:
import pandas as pd
import numpy as np

In [2]:
#Uncomment the line below to install stanza
#!pip install stanza

# Data collection

## <font color='green'>**Rerun the cell below**</font>

In [3]:
from util import spacy_get_sents, stanza_get_sents, spacy_tokenize_text, stanza_tokenize_text, get_texts

2023-04-26 01:08:55 INFO: Checking for updates to resources.json in case models have been updated.  Note: this behavior can be turned off with download_method=None or download_method=DownloadMethod.REUSE_RESOURCES


2023-04-26 01:08:56 INFO: Loading these models for language: en (English):
| Processor | Package  |
------------------------
| tokenize  | combined |
| pos       | combined |

2023-04-26 01:08:56 INFO: Using device: cpu
2023-04-26 01:08:56 INFO: Loading: tokenize
2023-04-26 01:08:56 INFO: Loading: pos
2023-04-26 01:08:56 INFO: Done loading processors!


For our further inverstigations we will choose 100 texts from "Writers" category that we previously stored:

In [20]:
texts = get_texts(source_type='folder', path='Writers', token_type='file', n=100)

Below we can see the sample of 5 files from the dataset:

In [21]:
texts[:5]

['Anne Von Bertouch, (29 June 1915 – 31 March 2003) was an Australian art dealer, author, environmentalist and director of the Von Bertouch Galleries in Newcastle, New South Wales, believed to be the first commercial gallery outside a capital city in Australia. Biography Born Anne Catherine, to parents Jean (née Duff) and George Whittle on 29 June 1915 in Eastwood, New South Wales, she was educated at Sydney Girls High School and Armidale Teachers College. After teacher training her first posting was at Adamstown Infants School in the 1930s. She married Roger Von Bertouch in 1939. In 1941 she was the organiser of a National Fitness Camp for girls at Broken Bay, New South Wales.She and Roger moved to Tasmania, where they taught and she studied at Hobart Technical College. In 1942 she performed modern interpretive dance at a Town Hall concert in Hobart organised by the Australian Broadcasting Commission Patriotic Committee as a fundraiser on Allies\' Appeal Day. She danced also in Hobart

# Sentence segmentation

For our selected 100 articles we will perform sentence tokenization using Spacy and Stanza.
Each file will be separated into sentences separately.

In [22]:
spacy_sents = [spacy_get_sents(x) for x in texts]

In [23]:
stanza_sents = [stanza_get_sents(x) for x in texts]

We can now show the number of sentences calculated by each library for our articles:

In [24]:
sents_df = pd.DataFrame(columns=['Spacy', 'Stanza'])

sents_df.Spacy = list(map(lambda x: len(x), spacy_sents))
sents_df.Stanza = list(map(lambda x: len(x), stanza_sents))

sents_df

,Spacy,Stanza
0,107,95
1,37,33
2,8,9
3,35,27
4,28,29
...,...,...
95,32,30
96,21,11
97,18,17
98,7,7


We can look at the stats for all articles, it appears that Spacy on average finds more sentences:

In [25]:
sents_df.describe()

,Spacy,Stanza
count,100.000000,100.000000
mean,30.770000,28.230000
std,26.211978,24.465129
min,3.000000,3.000000
25%,13.000000,11.000000
50%,25.000000,21.500000
75%,37.000000,33.250000
max,155.000000,143.000000


Below we can see the total number of sentences recognized by the both libraries:

In [26]:
sents_df.sum()

Spacy     3077
Stanza    2823
dtype: int64

Now we can find all unique sentences that both recognize.
First we can look at the number shared sentences per article:

In [27]:
# Make a set of unique sentences for each article for both Spacy and Stanza.
# Find the intersection = these are sentences recognized by both libraries.

shared_sents_per_article = list(map(lambda x,y: set(x).intersection(set(y)), spacy_sents, stanza_sents))

shared_sents_per_article_df = pd.DataFrame(columns=['shared_sents', 'count'])
shared_sents_per_article_df['shared_sents'] = shared_sents_per_article
shared_sents_per_article_df['count'] = [len(x) for x in shared_sents_per_article]
shared_sents_per_article_df

,shared_sents,count
0,"{Anne Von Bertouch, (29 June 1915 – 31 March 2...",68
1,"{2000., First published at the age of twenty, ...",24
2,"{In 2021, Our Life released a documentary, The...",7
3,"{Vertellingen uit den oorlog (1915), De muziek...",12
4,{He also wrote for several other publications ...,27
...,...,...
95,{This combination of views was due to his own ...,26
96,{The work is extremely significant from a hist...,10
97,{After the breakdown of the Peace of Amiens in...,12
98,"{The author of 270 articles, two monographs, o...",7


Below is the overall count of shared sentences:

In [28]:
total_sp_sent = sents_df.sum()['Spacy']
total_st_sent = sents_df.sum()['Stanza']
total_shared = shared_sents_per_article_df['count'].sum()

print(f"""
    Total Spacy sentences: {total_sp_sent}
    Total Stanza sentences: {total_st_sent}
    Shared sentences: {total_shared}
    Percentage of shared sentences for Spacy: {100*total_shared/total_sp_sent:.2f}%
    Percentage of shared sentences for Stanza: {100*total_shared/total_st_sent:.2f}%""")


    Total Spacy sentences: 3077
    Total Stanza sentences: 2823
    Shared sentences: 2263
    Percentage of shared sentences for Spacy: 73.55%
    Percentage of shared sentences for Stanza: 80.16%


We can also have a look at what different articles both libraries found looking at the first mismatched sentence pair 
for each article:

In [29]:
def get_first_mismatched_sent_pair(sp_sents, st_sents):
    n = len(sp_sents)
    m = len(st_sents)
    
    # Go through the array of sentences for both Spacy and Stanza till the end of smallest of them.
    for i in range(min(n, m)):
        # When we reach the first mismatch, print the sentences.
        if sp_sents[i] != st_sents[i]:
            print(f'Spacy sentence: {sp_sents[i]} \n\n'
            f'Stanza sentence: {st_sents[i]}\n'
            '_______________')
            return sp_sents[i], st_sents[i]
    
    # If there was no mismatch during the array traversal, then all the sentences were tokenized together.
    print('Text segmentized into sentences in the same way.\n_______________')
    return None

for i, pair in enumerate(zip(spacy_sents, stanza_sents)):
    print(f'Article: {i}\n\n')
    get_first_mismatched_sent_pair(pair[0], pair[1])

Article: 0


Spacy sentence: She danced also in Hobart's Opera And Ballet Festival for International Week in 1945.Intending to develop land and to pursue their artistic interests, they moved to Mungo Brush in the Myall Lakes, New South Wales, in 1951 or 1954, living a subsistence existence from prawn fishing and trading their home-grown produce, and were appointed Honorary Rangers there in 1955 under the Wild Flowers and Native Plants Protection Act. 

Stanza sentence: She danced also in Hobart's Opera And Ballet Festival for International Week in 1945.
_______________
Article: 1


Spacy sentence: ISBN 9782844096265, 

Stanza sentence: ISBN 9782844096265, ASIN B07TTX6ZHQ
_______________
Article: 2


Spacy sentence: He successfully underwent a ground-breaking bilateral arm transplant in August 2016.Peck wrote a book, Rebuilding Sergeant Peck: How I Put Body and Soul Back Together After Afghanistan, that was released on May 7, 2019. 

Stanza sentence: He successfully underwent a ground-b

From the sample we see above we can note the following behavior for sentence tokenization for Spacy and Stanza:
- If there is a missing space before a period, separating sentences, Stanza seems to perform sentence segmentation better, e.g `Article 0, 2, 7, 24`...
- In our source material some sections have a title which is not punctuated. Stanza appears to recognize such cases as a separate sentence (correct) more frequently, e.g. `Article 3, 6, 12, 16, 17, 19`...
- However, Stanza seems to be incorrectly splitting a sentence more frequently if there is a punctuation sign followed by a capital letter, e.g. `Article 4, 5, 9, 11, 15, 18`...
- Both libraries struggle (as expected) with sententences that have a lot of non-English words or the input string is not a correct sentence.

______________________

Now we can form a list of shared sentences for all articles combined:

In [30]:
# Convert the sets into arrays and then flatted the 2 array into 1-d array using numpy.concatenate

shared_sents = np.concatenate(list(map(lambda x: list(x), shared_sents_per_article)))

In [31]:
shared_sents[:5]

array(['Anne Von Bertouch, (29 June 1915 – 31 March 2003) was an Australian art dealer, author, environmentalist and director of the Von Bertouch Galleries in Newcastle, New South Wales, believed to be the first commercial gallery outside a capital city in Australia.',
       'ISBN 978-0-9592824-1-2. OCLC 27623615.',
       'Von Bertouch, Anne (1983).',
       'The terraces were purchased by Dr Dick Lees for #1,420,000 in 2007, and in 2010 were listed for sale at A$1.6 million.',
       'What was it before it was a gallery?.'], dtype='<U592')

We can now store the shared sentences into a separate DataFrame and into its own CSV-file:

In [32]:
shared_sents_df = pd.DataFrame(columns=['Sentence'])
shared_sents_df.Sentence = shared_sents
shared_sents_df

,Sentence
0,"Anne Von Bertouch, (29 June 1915 – 31 March 20..."
1,ISBN 978-0-9592824-1-2. OCLC 27623615.
2,"Von Bertouch, Anne (1983)."
3,The terraces were purchased by Dr Dick Lees fo...
4,What was it before it was a gallery?.
...,...
2258,"In popular culture In 1999, Komphet Phorncharo..."
2259,"In 1996, Butnakho started to write songs in th..."
2260,"The melody came from the pattern of ""Sao Simue..."
2261,"In 2007, Isan music became very popular in Lao..."


In [33]:
shared_sents_df.to_csv('SpacyStanza/spacy_stanza_shared_sents.csv')

# Tokenization

For the experiment below we will use all the tokens returned by Spacy and Stanza, without filtering any punctuation or spaces.

However, prior to all the analysis texts were cleaned: we removed linebreaks, tabulation and special characters like `|`, `^`, `<`, `>`, `+`, and `=`. These symbols were not returned when we requested the original texts.

We will do the final step of preprocessing and lowercase the text in order to avoid the same word being in vocabulary twice: for different spelling when it's in the beginning or in the middle of a sentence.

For our experiment we will take only the sentences that were previously segmented in the same way by Spacy and Stanza.
For **SharedTokensNosentences** we will stich all the sentences together and separate them by spaces. For **SharedTokensInSentences** we will apply tokenization on the sentences from `shared_sents_df` above.

## <font color='green'>**Rerun the 3 cells below**</font>

In [4]:
shared_sents_df = pd.read_csv('spacy_stanza_shared_sents.csv')

In [5]:
shared_sents_lower = [s.lower() for s in shared_sents_df.Sentence]

In [6]:
shared_sents_stiched = ' '.join(shared_sents_lower)

##  Vocabulary

To compare vocabularies we will run the comparison on the sentences stiched together into one text.

In [46]:
# Create a set to find unique tokens only
spacy_vocabulary = set(spacy_tokenize_text(shared_sents_stiched, no_filtering=True))

In [47]:
stanza_vocabulary = set(stanza_tokenize_text(shared_sents_stiched))

In [48]:
print(f'Spacy vocabulary size: {len(spacy_vocabulary)}\nStanza vocabulary size: {len(stanza_vocabulary)}')

Spacy vocabulary size: 8488
Stanza vocabulary size: 8526


### Shared vocabulary

Shared vocabulary is calculated as intersection of sets of Stanza and Spacy vocabulary:

In [50]:
shared_vocab = spacy_vocabulary.intersection(stanza_vocabulary)

In [51]:
shared_vocab_size = len(shared_vocab)

Size of shared vocabulary:

In [52]:
shared_vocab_size

8266

# <font color='red'>Please **DO NOT** rerun the cell below</font>

In [54]:
np.random.choice(list(shared_vocab), size=50)

array(['moving', 'specialized', 'there', 'ekspres', 'attended', 'pune',
       'foot', 'occasion', 'favored', '1961', '1659', 'believed', 'equal',
       'warsaw', 'turkish', 'height', 'growing', 'c.', 'having',
       'hinting', 'friends', 'sweetheart', 'organically', 'registry',
       'lhadj', 'nicholas', 'judgement', 'confinement', 'argentina',
       'pleasures', 'armidale', 'bunyan', 'immediate', 'emigrants',
       'ṛṛays', 'blessing', 'mae', 'extinct', 'mosques', 'stockholm',
       'stanitsa', 'matter', 'muffins', 'ceramics', 'egg', 'saitovic',
       'tourism', 'founded', 'hospitalized', 'an'], dtype='<U19')

From the sample above we can observe that both libraries seem to agree on tokenizing a wide range of words:
- Common words like `an`, `egg`, `there`, `having` are present in the shared vocabulary.
- Foreign to English words like `bunyan`, `stanitsa`, `lhadj` are also present in the shared vocabulary.
- We can as well see some Named Entities like dates and locations
- We can as well see an incorrect lemma `ṛṛays` (presumably a part of `arrays`) which was produced by both libraries.
- ...

### Spacy-only vocabulary

In [56]:
spacy_only_vocab = spacy_vocabulary.difference(stanza_vocabulary)
spacy_only_vocab

{"'m",
 "'ve",
 '):',
 ',(1898',
 '/priče',
 '/zapisi',
 '0',
 '040',
 '09',
 '1031–1034',
 '111',
 '1134068708',
 '1442252813',
 '1472806871',
 '1476679488',
 '1500',
 '1547–1564',
 '1689–1887',
 '16–17',
 '1744–1816',
 '1758',
 '1766–1822',
 '1766–1830',
 '1785–1863',
 '1793–1869',
 '1795–1864',
 '1798–1879',
 '1804–1856',
 '1804–1871',
 '1810–1860',
 '1848–1905',
 '1860–1923,1986',
 '1868–1936',
 '1877–1951',
 '1882–1956',
 '1885–1900',
 '1904–1920',
 '1905–1907',
 '1909–1910',
 '1909–1919',
 '1912–1928',
 '1913–1974',
 '1918–1923',
 '1928–1998',
 '1935–37',
 '1937–39',
 '1941–45',
 '1943–24',
 '1945–47',
 '1949–2009',
 '1950/1951',
 '1969–70',
 '1977–1980',
 '1980–84',
 '1989–1991',
 '1990–2008',
 '1996–2000',
 '1998–2002',
 '19–24',
 '2001–2011',
 '2005–2006',
 '2008–13',
 '2011/2012',
 '2018edited',
 '2019–20',
 '205',
 '23833',
 '255',
 '25–28',
 '293',
 '300p',
 '313',
 '323',
 '35–6',
 '3865',
 '3905881028',
 '39–40',
 '401',
 '441–2',
 '480–44',
 '521',
 '53',
 '605',
 '700–7

In the Spacy-only tokens we can see a lot of numbers as well as non-english words (or non-words).

However, surprisingly we can also see some more or less common English words:
- Some words of relatively more formal register of English: `canonical`, `colonialism`, `denominational`, `editors`, `eminent`, `feudal`, `imperialism`, `instrumentalist`...
- Some NE: `kishinev`, `kyoto`, `liechtenstein`, `vatican` (Locations), `suzuki` (Name or Organisation name), `unesco` (Organisation name)...

### Stanza-only vocubulary

In [57]:
stanza_only_vocab = stanza_vocabulary.difference(spacy_vocabulary)
stanza_only_vocab

{"'a.",
 "'e",
 "'t",
 '-1758',
 '-1980',
 '-2005',
 '-3865',
 '.ie',
 '/1951',
 '0-521-23833-1',
 '1-905237-53',
 '1031',
 '1689',
 '1744',
 '1766-1822',
 '1785',
 '1798',
 '1810',
 '1860',
 '1868',
 '1882',
 '1923,1986',
 '1923-2011',
 '1943-2012',
 '313-323',
 '39',
 '441',
 '47',
 '480',
 '700',
 '84',
 '84.5',
 '8527189.',
 '9695188.',
 '975-293-040-9',
 '975-293-255-x',
 '975-8084',
 '978-0-9592824-1-2.',
 '978-0-9592824-2-9.',
 '978-0-9592824-3-6.',
 '978-1134068708',
 '978-1442252813',
 '978-1472806871',
 '978-1476679488',
 '978-3-205-78442-5',
 '978-3-7349-9213-1',
 '978-3-7349-9223-0',
 '978-3-8392-1500',
 '978-3-8392-1978',
 '978-3-8392-1986-7',
 '978-3905881028',
 '978-605-09',
 '978-605-111-401-9',
 '978-976-8097-13-2.',
 '9781931883702',
 '?.',
 '[...]',
 '`the',
 '`yellow',
 "abc's",
 'ah.',
 'anti-apartheid',
 'anti-colonialism',
 'anti-feudal',
 'anti-german',
 'anti-imperialism',
 'anti-love',
 'anti-semite',
 'anti-semitic',
 'anti-semitism',
 'anti-soviet',
 'anti-v

We can see that Stanza tends tokenize words with prefixes like `co-`, `anti-`, `non-`, `neo-`... as one token, which explains the difference with Spacy on tokenization of words of relatively formal register.

We can se can see that there is the same tendency towards composed words: `pen-name`, `kingdom-based`, `serbo-croatian`...

As for Named Entities, there appears to be a lot of them tokenized together with `'s`: `yusuf's`, `unesco's`, `suzuki's`, `shin's`, `kyoto's`...

## Occurences

To calculate common accurances we will find all tokens produced by both models for a given string and will find tokens that have the same text for both Stanza and Spacy.

For this we will work with raw token entities from Stanza and Spacy to preserve PoS information for the following experiment since PoS information might be changed if a token is taken out of its context.

## <font color='green'>**Rerun the cell below**</font>

In [7]:
from util import get_tokens_dict, common_tokens

### Occurences without sentence separation

We will tokenize all stiched sentences together.

In [ ]:
no_sent_spacy_tokens = spacy_tokenize_text(shared_sents_stiched, to_string=False, no_filtering=True)

In [ ]:
no_sent_stanza_tokens = stanza_tokenize_text(shared_sents_stiched, to_string=False)

In [9]:
common_t = common_tokens(no_sent_spacy_tokens, no_sent_stanza_tokens)

In [10]:
common_t

[([erzählung],
  [[
     {
       "id": 1,
       "text": "erzählung",
       "start_char": 188945,
       "end_char": 188954
     }
   ]]),
 ([h., h.],
  [[
     {
       "id": 14,
       "text": "h.",
       "start_char": 75177,
       "end_char": 75179
     }
   ],
   [
     {
       "id": 4,
       "text": "h.",
       "start_char": 142563,
       "end_char": 142565
     }
   ]]),
 ([gabriëlla],
  [[
     {
       "id": 36,
       "text": "gabriëlla",
       "start_char": 9589,
       "end_char": 9598
     }
   ]]),
 ([internet, internet, internet],
  [[
     {
       "id": 18,
       "text": "internet",
       "start_char": 6606,
       "end_char": 6614
     }
   ],
   [
     {
       "id": 15,
       "text": "internet",
       "start_char": 180986,
       "end_char": 180994
     }
   ],
   [
     {
       "id": 15,
       "text": "internet",
       "start_char": 224673,
       "end_char": 224681
     }
   ]]),
 ([musa],
  [[
     {
       "id": 3,
       "text": "musa",
       "s

Some token stats:

In [11]:
print(f"""
Spacy tokens: {len(no_sent_spacy_tokens)}
Stanza tokens: {len(no_sent_stanza_tokens)}
Shared tokens: {len(np.concatenate([x[1] for x in common_t]))}
% of shared tokens in Spacy: {100*len(np.concatenate([x[1] for x in common_t]))/len(no_sent_spacy_tokens):.2f}%
% of shared tokens in Stanza: {100*len(np.concatenate([x[1] for x in common_t]))/len(no_sent_stanza_tokens):.2f}%

"""
)


Spacy tokens: 48820
Stanza tokens: 48502
Shared tokens: 48070
% of shared tokens in Spacy: 98.46%
% of shared tokens in Stanza: 99.11%




We will store the results in a dictionary where the key is the word form and the value is another dictionary, storing all seen results for Spacy and Stanza separately.

In [12]:
no_sent_shared_tokens = {x[0][0].text: {'Spacy': x[0], 'Stanza': x[1]} for x in common_t}

In [13]:
no_sent_shared_tokens

{'erzählung': {'Spacy': [erzählung],
  'Stanza': [[
     {
       "id": 1,
       "text": "erzählung",
       "start_char": 188945,
       "end_char": 188954
     }
   ]]},
 'h.': {'Spacy': [h., h.],
  'Stanza': [[
     {
       "id": 14,
       "text": "h.",
       "start_char": 75177,
       "end_char": 75179
     }
   ],
   [
     {
       "id": 4,
       "text": "h.",
       "start_char": 142563,
       "end_char": 142565
     }
   ]]},
 'gabriëlla': {'Spacy': [gabriëlla],
  'Stanza': [[
     {
       "id": 36,
       "text": "gabriëlla",
       "start_char": 9589,
       "end_char": 9598
     }
   ]]},
 'internet': {'Spacy': [internet, internet, internet],
  'Stanza': [[
     {
       "id": 18,
       "text": "internet",
       "start_char": 6606,
       "end_char": 6614
     }
   ],
   [
     {
       "id": 15,
       "text": "internet",
       "start_char": 180986,
       "end_char": 180994
     }
   ],
   [
     {
       "id": 15,
       "text": "internet",
       "start_char":

### Occurences with segmented sentences

Now we repeat the experiment but for the separated sentences above:

## <font color='green'>**Rerun ALL the cells below**</font>

In [8]:
sp_sent_tokens = list(map(lambda x: spacy_tokenize_text(x, no_filtering=True, to_string=False), shared_sents_lower))

In [9]:
st_sent_tokens = list(map(lambda x: stanza_tokenize_text(x, to_string=False), shared_sents_lower))

In [10]:
common_sent_tokens = list(map(lambda x: common_tokens(x[0], x[1]), zip(sp_sent_tokens, st_sent_tokens)))

In [11]:
sent_tokens_df = pd.DataFrame(columns=['Spacy_token_count', 
                                       'Stanza_token_count', 
                                       'Shared_token_count', 
                                       'Spacy_shared_percentage',
                                      'Stanza_shared_percentage'])

In [12]:
sent_tokens_df.Spacy_token_count = list(map(len, sp_sent_tokens))
sent_tokens_df.Stanza_token_count = list(map(len, st_sent_tokens))
sent_tokens_df.Shared_token_count = list(map(len, common_sent_tokens))
sent_tokens_df.Spacy_shared_percentage = 100 * sent_tokens_df.Shared_token_count / sent_tokens_df.Spacy_token_count 
sent_tokens_df.Stanza_shared_percentage = 100 * sent_tokens_df.Shared_token_count / sent_tokens_df.Stanza_token_count 

sent_tokens_df

,Spacy_token_count,Stanza_token_count,Shared_token_count,Spacy_shared_percentage,Stanza_shared_percentage
0,13,13,13,100.000000,100.000000
1,14,5,4,28.571429,80.000000
2,18,18,16,88.888889,88.888889
3,12,12,12,100.000000,100.000000
4,11,10,9,81.818182,90.000000
...,...,...,...,...,...
2258,10,10,10,100.000000,100.000000
2259,19,19,18,94.736842,94.736842
2260,16,16,14,87.500000,87.500000
2261,16,16,14,87.500000,87.500000


Below are average stats for tokens (how many tokens in a sentence are on average provided by Spacy, how many by Stanza, how many are shared etc.):

In [13]:
sent_tokens_df.mean()

Spacy_token_count           21.573133
Stanza_token_count          21.438356
Shared_token_count          18.313301
Spacy_shared_percentage     89.018023
Stanza_shared_percentage    89.536601
dtype: float64

Total number of shared tokens across all sentences:

In [14]:
sent_tokens_df.Shared_token_count.sum()

41443

We have less aligned tokens now, meaning that in the experiment above we might have matched tokens from different sentences.

We will combine all the shared tokens from our sentences into one dictionary of the same structure as above: the keys are wordforms, the values are dictionaries containing arrays of all of the occurences of the wordforms.

Since for PoS analysis for Stanza requires words instead of tokens, we will convert the conversion as well.

In [30]:
shared_sent_tokens = {}

for sent in common_sent_tokens:
    for word in sent:
        word_text = word[0][0].text
        word_dict = shared_sent_tokens.get(word_text, {})
        
        sp_t_list = word_dict.get('Spacy', [])
        sp_t_list.extend(word[0])
        word_dict['Spacy'] = sp_t_list
        
        st_t_list = word_dict.get('Stanza', [])
        # Before extending the list convert tokens to words
        st_t_list.extend(list(map(lambda x: x.words[0], word[1])))
        word_dict['Stanza'] = st_t_list
        
        shared_sent_tokens[word_text] = word_dict


Below is an example of how the dictionary will work on an example of one word `chief`.

`shared_sent_tokens['chief']['Spacy']` can be called to get all Spacy tokens and similarly `shared_sent_tokens['chief']['Stanza']` can be called to get all Stanza tokens.

In this example we can see that this word was sometimes categorized as Noun and sometimes as Adjective.

In [32]:
shared_sent_tokens['chief']

{'Spacy': [chief,
  chief,
  chief,
  chief,
  chief,
  chief,
  chief,
  chief,
  chief,
  chief,
  chief,
  chief,
  chief,
  chief,
  chief],
 'Stanza': [{
    "id": 12,
    "text": "chief",
    "upos": "ADJ",
    "xpos": "JJ",
    "feats": "Degree=Pos",
    "start_char": 68,
    "end_char": 73
  },
  {
    "id": 5,
    "text": "chief",
    "upos": "ADJ",
    "xpos": "JJ",
    "feats": "Degree=Pos",
    "start_char": 20,
    "end_char": 25
  },
  {
    "id": 6,
    "text": "chief",
    "upos": "ADJ",
    "xpos": "JJ",
    "feats": "Degree=Pos",
    "start_char": 17,
    "end_char": 22
  },
  {
    "id": 20,
    "text": "chief",
    "upos": "NOUN",
    "xpos": "NN",
    "feats": "Number=Sing",
    "start_char": 100,
    "end_char": 105
  },
  {
    "id": 5,
    "text": "chief",
    "upos": "NOUN",
    "xpos": "NN",
    "feats": "Number=Sing",
    "start_char": 26,
    "end_char": 31
  },
  {
    "id": 21,
    "text": "chief",
    "upos": "NOUN",
    "xpos": "NN",
    "feats": "Number

How PoS can be accessed for Spacy:

In [33]:
shared_sent_tokens['chief']['Spacy'][0], shared_sent_tokens['chief']['Spacy'][0].pos_

(chief, 'ADJ')

How PoS can be accessed for Stanza:

In [35]:
shared_sent_tokens['chief']['Stanza'][0], shared_sent_tokens['chief']['Stanza'][0].upos

({
   "id": 12,
   "text": "chief",
   "upos": "ADJ",
   "xpos": "JJ",
   "feats": "Degree=Pos",
   "start_char": 68,
   "end_char": 73
 },
 'ADJ')

# Part of Speech